In [ ]:
from transformers import AutoImageProcessor, SiglipForImageClassification
from pathlib import Path
from tqdm import tqdm
from PIL import Image
import torch
import pandas as pd

# === Model Setup ===
# Load the pretrained facial age classification model and its image processor.
# The model is used in evaluation (inference) mode only, so no gradients are computed.
MODEL_NAME = "prithivMLmods/facial-age-detection"

print(f"Loading model: {MODEL_NAME}")
processor = AutoImageProcessor.from_pretrained(MODEL_NAME)
model = SiglipForImageClassification.from_pretrained(MODEL_NAME)
model.eval()  # ensures dropout, etc., are disabled for prediction
print("Model loaded successfully!")


⏳ Loading model: prithivMLmods/facial-age-detection
✅ Model loaded successfully!


In [ ]:
# Mapping from model output class indices to human-readable age ranges.
# The model returns a numerical class ID; this dictionary converts it into an age bracket label.
id2label = {
    "0": "01–10",
    "1": "11–20",
    "2": "21–30",
    "3": "31–40",
    "4": "41–55",
    "5": "56–65",
    "6": "66–80",
    "7": "80+"
}


In [ ]:
# Define input and output locations.
# INPUT_DIR: Folder containing the unique, final face images (one per cluster).
# CSV_PATH:  Where the age prediction results will be saved as a CSV file.
# OUTPUT_DIR: Folder where annotated age prediction images will be written.
INPUT_DIR = Path("../data/crops_face/20251105/5_cafe_pos_faces_unique")
CSV_PATH = Path("../data/predictions/cafe_pos/20251105/age_annotated/age_predictions.csv")
OUTPUT_DIR = Path("../data/predictions/cafe_pos/20251105/age_annotated")

# Ensure that the output directory exists before saving annotated images or the result CSV.
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


In [ ]:
# Prepare list to store all prediction results.
results = []

# Display which folder is being scanned for input images.
# INPUT_DIR should contain the unique frontal face crops (one per detected customer).
print(f" Scanning folder: {INPUT_DIR}")

# Collect all image files in the directory.
# We sort them to ensure consistent and reproducible processing order.
# Only common image extensions are included.
image_paths = sorted([
    p for p in INPUT_DIR.glob("*.*")
    if p.suffix.lower() in [".jpg", ".png", ".jpeg"]
])

# Report how many input images were found for age prediction.
print(f" Found {len(image_paths)} face crops.\n")


📂 Scanning folder: ..\data\crops_face\20251105\5_cafe_pos_faces_unique
🧠 Found 287 face crops.



In [ ]:
# Iterate through all detected face images and generate age predictions for each.
for img_path in tqdm(image_paths, desc="Predicting age groups"):
    try:
        # Load the image and convert to RGB to ensure compatibility with the model.
        image = Image.open(img_path).convert("RGB")

        # Preprocess the image using the model's processor to produce tensors.
        inputs = processor(images=image, return_tensors="pt")

        # Perform a forward pass through the model without gradient tracking.
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits

            # Convert the raw logits into a probability distribution.
            probs = torch.nn.functional.softmax(logits, dim=1).squeeze().tolist()

        # Determine the index of the class with the highest predicted probability.
        top_idx = int(torch.argmax(torch.tensor(probs)))

        # Translate the class index into the corresponding age range label.
        predicted_age_group = id2label[str(top_idx)]

        # Extract the confidence score associated with the prediction.
        confidence = round(probs[top_idx], 3)

        # Append all prediction details for this image to the results list.
        results.append({
            "file": img_path.name,
            "predicted_age": predicted_age_group,
            "confidence": confidence,
            "probs": {id2label[str(i)]: round(p, 3) for i, p in enumerate(probs)}
        })

    except Exception as e:
        # Log problematic images and continue processing the remainder.
        print(f"Skipped {img_path.name} — error: {e}")
        continue


Predicting age groups: 100%|██████████| 287/287 [01:29<00:00,  3.19it/s]


In [ ]:
# Convert the collected prediction results into a DataFrame for structured analysis.
df_results = pd.DataFrame(results)

# Export the DataFrame to a CSV file so results can be reused in later stages.
df_results.to_csv(CSV_PATH, index=False)

# Confirm that the file has been written to disk.
print(f"\nSaved predictions to: {CSV_PATH}")

# Display a preview of the first rows for verification.
display(df_results.head(70))



✅ Saved predictions to: ..\data\predictions\cafe_pos\20251105\age_annotated\age_predictions.csv


,file,predicted_age,confidence,probs
0,cluster_000_AccoID_1169434_20251105_131026.png,21–30,0.443,"{'01–10': 0.066, '11–20': 0.196, '21–30': 0.44..."
1,cluster_001_AccoID_1168923_20251105_105123.png,66–80,0.305,"{'01–10': 0.015, '11–20': 0.118, '21–30': 0.01..."
2,cluster_002_AccoID_1169467_20251105_132245.png,56–65,0.566,"{'01–10': 0.004, '11–20': 0.005, '21–30': 0.00..."
3,cluster_003_AccoID_1168748_20251105_095655.png,66–80,0.776,"{'01–10': 0.0, '11–20': 0.003, '21–30': 0.001,..."
4,cluster_004_AccoID_1168916_20251105_104927.png,56–65,0.400,"{'01–10': 0.001, '11–20': 0.001, '21–30': 0.03..."
...,...,...,...,...
65,cluster_066_AccoID_1169213_20251105_121321.png,66–80,0.696,"{'01–10': 0.0, '11–20': 0.0, '21–30': 0.0, '31..."
66,cluster_067_AccoID_1169220_20251105_121427.png,80+,0.563,"{'01–10': 0.0, '11–20': 0.0, '21–30': 0.0, '31..."
67,cluster_068_AccoID_1168847_20251105_103500.png,56–65,0.693,"{'01–10': 0.001, '11–20': 0.003, '21–30': 0.00..."
68,cluster_069_AccoID_1169227_20251105_121648.png,80+,0.498,"{'01–10': 0.0, '11–20': 0.0, '21–30': 0.0, '31..."


In [ ]:
import os
import cv2
import pandas as pd
from pathlib import Path

# === Paths ===
# Directory containing the original unique face crops.
INPUT_DIR = Path("../data/crops_face/20251105/5_cafe_pos_faces_unique")

# CSV file containing age prediction results produced by the model.
CSV_PATH = Path("../data/predictions/cafe_pos/20251105/age_annotated/age_predictions.csv")

# Output directory for storing the age-annotated images.
OUTPUT_DIR = Path("../data/predictions/cafe_pos/20251105/age_annotated")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# === Load Predictions ===
# Load the CSV so each image can be annotated with its predicted age group and confidence.
df_results = pd.read_csv(CSV_PATH)
print(f"Loaded {len(df_results)} predictions from {CSV_PATH}")

# === Style Settings ===
# These define how the label text is drawn on the image: font, size, color, background, and transparency.
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.7
thickness = 2
text_color = (255, 255, 255)
bg_color = (0, 0, 0)
opacity = 0.5

# === Annotate and Save ===
# Loop through each prediction, load the corresponding image, draw the label overlay, and save it.
for _, row in df_results.iterrows():

    filename = row["file"]
    predicted_age = str(row["predicted_age"])
    confidence = row.get("confidence", None)

    src_path = INPUT_DIR / filename
    dst_path = OUTPUT_DIR / filename

    img = cv2.imread(str(src_path))
    if img is None:
        print(f"Skipping missing image: {filename}")
        continue

    # Construct the text label, optionally including confidence.
    if confidence is not None:
        text = f"{predicted_age} ({confidence:.2f})"
    else:
        text = predicted_age

    # Remove unsupported characters to avoid OpenCV rendering issues.
    text = text.replace("–", "-").encode("ascii", errors="ignore").decode()

    # Draw a semi-transparent rectangle as the label background.
    (tw, th), _ = cv2.getTextSize(text, font, font_scale, thickness)
    overlay = img.copy()
    cv2.rectangle(overlay, (10, 10), (10 + tw + 20, 10 + th + 20), bg_color, -1)
    img = cv2.addWeighted(overlay, opacity, img, 1 - opacity, 0)

    # Render the text label onto the image.
    cv2.putText(img, text, (20, 10 + th + 8), font, font_scale, text_color, thickness, cv2.LINE_AA)

    # Save the annotated image to the output directory.
    cv2.imwrite(str(dst_path), img)

print(f"\nAnnotated images saved to: {OUTPUT_DIR}")


📄 Loaded 287 predictions from ..\data\predictions\cafe_pos\20251105\age_annotated\age_predictions.csv

✅ Annotated images saved to: ..\data\predictions\cafe_pos\20251105\age_annotated


In [ ]:
import pandas as pd

# === Paths ===
# Path to the age prediction results produced by the age classification model.
age_csv = "../data/predictions/cafe_pos/20251105/age_annotated/age_predictions.csv"

# Path to the latest merged dataset, which already contains face metadata and gender predictions.
results_stage2 = "../results/20251105/results_faces_accounting_20251105.csv"

# Path where the new merged dataset (including age information) will be written.
output_csv = "../results/20251105/results_faces_accounting_20251105.csv"

# === Load Data ===
# Load both age predictions and existing merged results into memory.
age_df = pd.read_csv(age_csv)
results_df = pd.read_csv(results_stage2)

print("Loaded files:")
print(f" - Age predictions: {len(age_df)} rows")
print(f" - Current results (with gender): {len(results_df)} rows\n")

# === Standardize Column Names ===
# Ensure both datasets reference the image filename in a consistent format.
age_df.rename(columns={"file": "image_name"}, inplace=True)

# Remove directory prefixes if any exist; keep only the filename.
age_df["image_name"] = age_df["image_name"].apply(lambda x: x.split("/")[-1])
results_df["image_name"] = results_df["image_name"].apply(lambda x: x.split("/")[-1])

# === Merge on Image Name ===
# Join the age predictions onto the existing dataset using the shared image filename.
merged_df = results_df.merge(
    age_df[["image_name", "predicted_age", "confidence"]],
    on="image_name",
    how="left",
    suffixes=("", "_age")
)

# If both gender and age confidence columns exist, rename them for clarity.
if "confidence_age" in merged_df.columns:
    merged_df.rename(columns={
        "confidence": "gender_confidence",
        "confidence_age": "age_confidence"
    }, inplace=True)
else:
    # If only gender confidence exists, rename it to avoid ambiguity.
    merged_df.rename(columns={"confidence": "gender_confidence"}, inplace=True)

# === Save Final Output ===
# Write the fully merged dataset (faces + accounting + gender + age) to disk.
merged_df.to_csv(output_csv, index=False)
print(f"Stage 3 results (with age and gender) saved to: {output_csv}")

# Display a preview of the merged dataset.
merged_df.head()


✅ Loaded files:
 - Age predictions: 287 rows
 - Current results (with gender): 796 rows

✅ Stage 3 results (with age + gender) saved to: ../results/20251105/results_faces_accounting_20251105.csv


,image_name,cluster_id,AccoID,AccoDocNo,AccoDate,timestamp,AccoAmount,Quantity,Discount,UnitPrice,StockCateDesc,StockName,StockDesciption,predicted_gender,gender_confidence,predicted_age,age_confidence
0,cluster_000_AccoID_1169434_20251105_131026.png,0,1169434,224673/155,2025-11-05 13:10:00,2025-11-05 13:10:26,-85.0,1.0,0.0,29.0,Mineral cooldrinks,COKE,original,Male,0.946,21–30,0.443
1,cluster_000_AccoID_1169434_20251105_131026.png,0,1169434,224673/155,2025-11-05 13:10:00,2025-11-05 13:10:26,-85.0,1.0,0.0,20.0,Beverages Cold,WATER,thirsty mineral still,Male,0.946,21–30,0.443
2,cluster_000_AccoID_1169434_20251105_131026.png,0,1169434,224673/155,2025-11-05 13:10:00,2025-11-05 13:10:26,-85.0,1.0,0.0,36.0,Beverages Coffee,COFFEE,cappuccino,Male,0.946,21–30,0.443
3,cluster_001_AccoID_1168923_20251105_105123.png,1,1168923,224590/74,2025-11-05 10:52:00,2025-11-05 10:51:23,-256.0,1.0,0.0,29.0,Mineral cooldrinks,COKE,original,Female,0.908,66–80,0.305
4,cluster_001_AccoID_1168923_20251105_105123.png,1,1168923,224590/74,2025-11-05 10:52:00,2025-11-05 10:51:23,-256.0,1.0,0.0,59.0,Breakfast,BREAKFAST,lite,Female,0.908,66–80,0.305


In [ ]:
# ==============================================
# PHASE 1: Run the ViT Age Classifier
# ==============================================
import torch
from transformers import ViTFeatureExtractor, ViTForImageClassification
from pathlib import Path
from tqdm import tqdm
from PIL import Image
import pandas as pd
import numpy as np

# --- Model setup ---
# Load the pre-trained ViT age classifier along with its feature extractor.
# The model outputs logits corresponding to predefined age-range categories.
MODEL_NAME = "nateraw/vit-age-classifier"
print(f"Loading model: {MODEL_NAME}")

model = ViTForImageClassification.from_pretrained(MODEL_NAME)
feature_extractor = ViTFeatureExtractor.from_pretrained(MODEL_NAME)
model.eval()
print("Model loaded successfully.\n")

# --- Paths ---
# INPUT_DIR: Directory containing the detected frontal face crops.
# OUTPUT_DIR: Where raw ViT age predictions will be stored.
# CSV_PATH: CSV file capturing the prediction outputs for later stages.
INPUT_DIR = Path("../data/crops_face/20251107/5_cafe_pos_faces_unique")
OUTPUT_DIR = Path("../data/predictions/cafe_pos/20251107/age_vit_raw")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
CSV_PATH = OUTPUT_DIR / "age_predictions_vit_raw.csv"

# --- Model labels (from ViT) ---
# These are the age groups used by the pre-trained classifier.
AGE_LABELS = [
    "0-2", "3-9", "10-19", "20-29",
    "30-39", "40-49", "50-59", "60-69", "more than 70"
]

# --- Run inference ---
# Iterate through each face image, prepare it for the model, obtain the prediction,
# and record the highest-probability age category.
results = []
image_paths = sorted([p for p in INPUT_DIR.glob("*.*") if p.suffix.lower() in [".jpg", ".png", ".jpeg"]])

print(f"Found {len(image_paths)} face crops.\n")

for img_path in tqdm(image_paths, desc="Predicting age groups (ViT)"):
    try:
        image = Image.open(img_path).convert("RGB")
        inputs = feature_extractor(images=image, return_tensors="pt")

        with torch.no_grad():
            outputs = model(**inputs)
            probs = torch.nn.functional.softmax(outputs.logits, dim=1).squeeze().tolist()

        top_idx = int(np.argmax(probs))
        predicted_age_raw = AGE_LABELS[top_idx]
        confidence = round(probs[top_idx], 3)

        results.append({
            "file": img_path.name,
            "predicted_age_raw": predicted_age_raw,
            "confidence_raw": confidence
        })

    except Exception as e:
        print(f"Skipped {img_path.name} — {e}")
        continue

# Convert the results to a DataFrame and save them for later processing.
df_raw = pd.DataFrame(results)
df_raw.to_csv(CSV_PATH, index=False)

print(f"\nSaved raw ViT predictions to: {CSV_PATH}")
display(df_raw.head())


⏳ Loading model: nateraw/vit-age-classifier


c:\Users\mrosk\anaconda3\envs\env_age_models\lib\site-packages\transformers\models\vit\feature_extraction_vit.py:30: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


✅ Model loaded successfully!

📂 Found 235 face crops.



Predicting age groups (ViT): 100%|██████████| 235/235 [01:10<00:00,  3.33it/s]


✅ Saved raw ViT predictions to: ..\data\predictions\cafe_pos\20251107\age_vit_raw\age_predictions_vit_raw.csv


,file,predicted_age_raw,confidence_raw
0,cluster_000_AccoID_1170854_20251107_091623.png,20-29,0.742
1,cluster_001_AccoID_1171262_20251107_112821.png,30-39,0.448
2,cluster_002_AccoID_1171659_20251107_135038.png,20-29,0.543
3,cluster_004_AccoID_1171659_20251107_135011.png,20-29,0.396
4,cluster_006_AccoID_1171574_20251107_132453.png,10-19,0.491


In [ ]:
# ==============================================
# PHASE 2: Re-bin ViT predictions
# ==============================================
import pandas as pd
from pathlib import Path

# Paths to raw ViT predictions and to the rebinned output file
RAW_PATH = "../data/predictions/cafe_pos/20251107/age_vit_raw/age_predictions_vit_raw.csv"
REBIND_PATH = "../data/predictions/cafe_pos/20251107/age_vit_raw/age_vit_rebinned.csv"

# Load raw age predictions from the ViT model
df = pd.read_csv(RAW_PATH)
print(f"Loaded {len(df)} raw predictions from ViT model")

# --- Map ViT labels to thesis-style bins ---
# Convert the ViT age ranges into the coarser age brackets used in the thesis.
BIN_MAP = {
    "0-2": "01–10",
    "3-9": "01–10",
    "10-19": "11–20",
    "20-29": "21–30",
    "30-39": "31–40",
    "40-49": "41–55",
    "50-59": "41–55",
    "60-69": "56–65",
    "more than 70": "66-80"  # Adjust to "80+" if that better matches your final binning
}

# Apply re-binning to the raw predicted label
df["predicted_age"] = df["predicted_age_raw"].map(BIN_MAP)

# Any label that does not map successfully falls back to "Unknown"
df["predicted_age"].fillna("Unknown", inplace=True)

# Rename confidence field for clarity and consistency
df["age_confidence"] = df["confidence_raw"]

# --- Save re-binned predictions ---
Path(REBIND_PATH).parent.mkdir(parents=True, exist_ok=True)
df.to_csv(REBIND_PATH, index=False)
print(f"Saved rebinned age predictions to: {REBIND_PATH}")

# --- Quick summary ---
print("\nAge group distribution:")
print(df["predicted_age"].value_counts())
df.head()


✅ Loaded 235 raw predictions from ViT model
✅ Saved rebinned age predictions to: ../data/predictions/cafe_pos/20251107/age_vit_raw/age_vit_rebinned.csv

📊 Age Group Distribution:
predicted_age
21–30    67
41–55    53
56–65    41
11–20    40
31–40    28
66-80     3
01–10     3
Name: count, dtype: int64


C:\Users\mrosk\AppData\Local\Temp\ipykernel_28096\2782564169.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["predicted_age"].fillna("Unknown", inplace=True)


,file,predicted_age_raw,confidence_raw,predicted_age,age_confidence
0,cluster_000_AccoID_1170854_20251107_091623.png,20-29,0.742,21–30,0.742
1,cluster_001_AccoID_1171262_20251107_112821.png,30-39,0.448,31–40,0.448
2,cluster_002_AccoID_1171659_20251107_135038.png,20-29,0.543,21–30,0.543
3,cluster_004_AccoID_1171659_20251107_135011.png,20-29,0.396,21–30,0.396
4,cluster_006_AccoID_1171574_20251107_132453.png,10-19,0.491,11–20,0.491


In [ ]:
# ==============================================
# PHASE 3: Merge rebinned ViT age predictions with existing gender results
# ==============================================
import pandas as pd
from pathlib import Path

# --- Inputs ---
# results_in:  dataset that already includes face metadata and gender predictions
# age_rebinned: rebinned ViT age predictions generated in the previous phase
# results_out: output path for the updated merged dataset
results_in  = "../results/20251105/results_faces_accounting_20251105_vit.csv"
age_rebinned = "../data/predictions/cafe_pos/20251105/age_vit_raw/age_vit_rebinned.csv"
results_out = "../results/20251105/results_faces_accounting_20251105_vit.csv"

# --- Load datasets ---
# Load both the existing results (containing gender) and the rebinned age predictions.
age_df = pd.read_csv(age_rebinned)
results_df = pd.read_csv(results_in)

print("Loaded files:")
print(f" - ViT age predictions: {len(age_df)} rows")
print(f" - Existing results:    {len(results_df)} rows\n")

# --- Standardize keys ---
# Ensure both DataFrames use the same filename format to enable a correct merge.
age_df.rename(columns={"file": "image_name"}, inplace=True)
age_df["image_name"] = age_df["image_name"].astype(str).str.split("/").str[-1]
results_df["image_name"] = results_df["image_name"].astype(str).str.split("/").str[-1]

# --- Merge ---
# Attach age predictions to the existing dataset using `image_name` as the key.
merged = results_df.merge(
    age_df[["image_name", "predicted_age", "age_confidence"]],
    on="image_name",
    how="left"
)

# --- Save merged dataset ---
Path(results_out).parent.mkdir(parents=True, exist_ok=True)
merged.to_csv(results_out, index=False)
print(f"Final merged dataset saved to:\n{results_out}")

# --- Summary check ---
n = len(merged)
filled = merged["predicted_age"].notna().sum()
print(f"Added new age predictions for {filled}/{n} rows ({filled/n*100:.2f}%)")

display(merged.head(10))


✅ Loaded files:
 - ViT age predictions: 287 rows
 - Existing results:    796 rows

✅ Final merged dataset saved to:
../results/20251105/results_faces_accounting_20251105_vit.csv
ℹ️ Added new age predictions for 796/796 rows (100.00%)


,image_name,cluster_id,AccoID,AccoDocNo,AccoDate,timestamp,AccoAmount,Quantity,Discount,UnitPrice,StockCateDesc,StockName,StockDesciption,predicted_gender,confidence,predicted_age,age_confidence
0,cluster_000_AccoID_1169434_20251105_131026.png,0,1169434,224673/155,2025-11-05 13:10:00,2025-11-05 13:10:26,-85.0,1.0,0.0,29.0,Mineral cooldrinks,COKE,original,Male,0.946,31–40,0.685
1,cluster_000_AccoID_1169434_20251105_131026.png,0,1169434,224673/155,2025-11-05 13:10:00,2025-11-05 13:10:26,-85.0,1.0,0.0,20.0,Beverages Cold,WATER,thirsty mineral still,Male,0.946,31–40,0.685
2,cluster_000_AccoID_1169434_20251105_131026.png,0,1169434,224673/155,2025-11-05 13:10:00,2025-11-05 13:10:26,-85.0,1.0,0.0,36.0,Beverages Coffee,COFFEE,cappuccino,Male,0.946,31–40,0.685
3,cluster_001_AccoID_1168923_20251105_105123.png,1,1168923,224590/74,2025-11-05 10:52:00,2025-11-05 10:51:23,-256.0,1.0,0.0,29.0,Mineral cooldrinks,COKE,original,Female,0.908,11–20,0.579
4,cluster_001_AccoID_1168923_20251105_105123.png,1,1168923,224590/74,2025-11-05 10:52:00,2025-11-05 10:51:23,-256.0,1.0,0.0,59.0,Breakfast,BREAKFAST,lite,Female,0.908,11–20,0.579
5,cluster_001_AccoID_1168923_20251105_105123.png,1,1168923,224590/74,2025-11-05 10:52:00,2025-11-05 10:51:23,-256.0,1.0,0.0,129.0,Breakfast,BREAKFAST,avo full house,Female,0.908,11–20,0.579
6,cluster_001_AccoID_1168923_20251105_105123.png,1,1168923,224590/74,2025-11-05 10:52:00,2025-11-05 10:51:23,-256.0,1.0,0.0,39.0,Beverages Hot,CHAI LATTE,none,Female,0.908,11–20,0.579
7,cluster_002_AccoID_1169467_20251105_132245.png,2,1169467,224678/161,2025-11-05 13:24:00,2025-11-05 13:22:45,-104.0,1.0,0.0,36.0,Beverages Coffee,COFFEE,cappuccino,Male,0.935,41–55,0.300
8,cluster_002_AccoID_1169467_20251105_132245.png,2,1169467,224678/161,2025-11-05 13:24:00,2025-11-05 13:22:45,-104.0,1.0,0.0,29.0,Beverages Coffee,COFFEE,americano,Male,0.935,41–55,0.300
9,cluster_002_AccoID_1169467_20251105_132245.png,2,1169467,224678/161,2025-11-05 13:24:00,2025-11-05 13:22:45,-104.0,1.0,0.0,39.0,Cakes,LEMON DRIZZLE,none,Male,0.935,41–55,0.300
